### 강아지와 고양이 사진 분류 모델의 성능 개선하기 목차

* [Chapter 1 잔차 블록(residual block)](#chapter1)
* [Chapter 2 ReNet 모델 만들기](#chapter2)
   * [Section 2.1 잔차 블록 만들기](#section_2_1)
   * [Section 2.2 잔차 스택 만들기](#section_2_2)
   * [Section 2.3 ReNet 모델 만들기](#section_2_3)
* [Chapter 3 강아지와 고양이 사진 분류하기](#chapter3)

### Chapter 1 잔차 블록(residual block) <a class="anchor" id="chapter1"></a>
1. 딥러닝 모델의 훈련과정
   - 경사 하강법으로 신경망 훈련 --> 손실 함수를 통해 모델의 출력과 타깃 사의의 오차 계산
     --> 모델의 끝에서부터 앞으로 누적하여 계산하는 역전파
   - 역전파: 출력 값과 실제 값의 차이를 확인하는 과정에서 오차 발새의 원인을 찾고, 신경망의 가중치를 조정하는 역활을 한다.

2. 딥러닝 모델은 신경망이 깊어질수록 이 가중치를 변경해 오차을 줄이는 그레디언트가 점점 작아진다.
   - 입력 부분에 가까운 가중치가 잘 변경하지 않는 문제가 발생한다.

3. 잔차 블록(residual block)은 입력을 출력에 직접 연결하는 스킵 연결을 추가해 그레디언트 소실을 완화한다.
   - 신경망의 층이 깊어지더라도 그레디언트가 잘 전파되어 신경망의 모든 층이 잘 훈련되는 효과를 낸다.

   ![잔차 블록](image/02-03-sidualBlock.png)    

4. ReNet50, ReNet101, ReNet152에서 사용하는 잔차 블록은 ReNet18, ReNet345의 잔차 블록과 좀 다르다.
   - 합성곱 층 다음에 배치 정규화가 추가된다.
   - 모델의 연산량을 줄이기 위해 3개의 합성곱층으로 구성된 병목 블록(bottleneck block)을 구성한다.
      - 첫 번재 합성곱층: 입력의 공간 방향의 크기를 출인다.
      - 두 번째 합성곱층: 입력 채널의 크기를 유지한 채 특징을 추출한다.
      - 세 번재 합성곱층: 채널의 수를 확장한다.


### Chapter 2 ReNet 모델 만들기 <a class="anchor" id="chapter2"></a>
1. ReNet 모델은 잔차 모듈이라는 구조를 통해 더욱 깊은 신경망에서도 모델의 학습 성능을 향상시킨다.

2. 잔차 모듈은 스킵연결을 통해 입력 데이터를 직접 다음 층으로 전달한다.
   - 신경망이 깊어지더라도 학습이 가능하도록 돕는다.
   - 잔차 블록과 그 잔차 블록이 모인 잔차 스텍으로 구성된다.

3. 배치 정규화
   - 잔차 블록 내에서 학습의 속도를 높이고 모델의 안전성을 개선하기위해 사용한다.
   - 신경망 입력은 보통 표준화를 통해 평균이 0, 분산이 1이 되도록 정규화된다.
   - 입력 데이터가 신경망의 여러층을 통과하면서 이런 정규화가 깨질 수 있다.
   - 배치 정규화 층은 배치 단위로 다시 정규화 함으로써 훈련의 속도와 성능을 높인다.
   - 샘플의 평균을 계산한다.
      - m: 배치에 있는 샘플 개수, B: 배치에 대한 평균임을 표시

         ![Avg](image/02-03-avg3.png) 

   - 분산을 계산한다.
      - B: 배치에 대한 분산임을 표시

         ![분산](image/02-03-var.png) 

   - 정규화를 진행한다.
      - 입력에서 평균을 빼고 표준 편차로 나눈다.
      - 분모가 0이되지 않도록 아주 작은 입실론(∈: 1*10^-5, 기본값 0.001)을 더한다.

          ![표준화](image/02-03-stand.png) 

   - 정규화가 진행된 값에 감와 베타를 더해 최종 출력을 만든다.
      - 정규화된 값은 평균이 0, 표준 편차가 1이므로 시그모이드 곡선의 직선에 대부분 위치하게되어 효과가 줄어든다.
      - 감마와 베타를 더하여 평균값의 범위를 변경한다.
      - 감마와 베타값도 역전파를 통해 학습된다.

         ![표준화2](image/02-03-stand2.png)
   
         
4. ReNet 전체구조   
   ![ReNet](image/02-03-ReNet.png)    

#### 2.1 잔차 블록 만들기 <a class="anchor" id="section_2_1"></a>

  ![잔차 블록2](image/02-03-sidualBlock2.png)    


In [4]:
import keras 
from keras import layers

# 입력 크기 정의
inputs = layers.Input(shape=(224, 224, 3))

# 케라스 합성곱층과 플링층의 패딩은 'valid'와 'same' 두 가지만 사용가능
#   - 원하는 크기로 패딩을 추가하려면 ZeroPadding2D 층을 사용해야 함
# padding=3 : 이미지 위/아래 각각 3픽셀, 좌/우 각각 3픽셀씩 0으로 채움
# 출력 크기 : (224 + 3*2, 224 + 3*2) = (230, 230)
x = layers.ZeroPadding2D(padding=3)(inputs)

#7*7 필터 64개, 스트라이드 2 합성곱 층에 입력 데이터 x 전달
# 출력 크기 : ( (230 - 7) / 2 + 1, (230 - 7) / 2 + 1 ) = (112, 112)
x = layers.Conv2D(64, kernel_size=7, strides=2)(x)

# 배치 정규화 층에 입력 데이터 x 전달
#   - epsilon : 분모가 0이 되는 것을 방지하기 위한 아주 작은 값을 분산에 더한다.
x = layers.BatchNormalization(epsilon=1e-5)(x)
x = layers.Activation('relu')(x)

# padding=1 : 이미지 위/아래 각각 1픽셀, 좌/우 각각 1픽셀씩 0으로 채움
# 출력 크기 : (112 + 1*2, 112 + 1*2) = (114, 114)
x = layers.ZeroPadding2D(padding=1)(x)\

# 3*3 필터, 스트라이드 2 맥스풀링 층에 입력 데이터 x 전달
# 출력 크기 : ( (114 - 3) / 2 + 1, (114 - 3) / 2 + 1 ) = (56, 56)
x = layers.MaxPooling2D(pool_size=3, strides=2)(x)

#### 2.2 잔차 스택 만들기 <a class="anchor" id="section_2_2"></a>
1. ReNet 모델의 핵심 부분은 4개의 잔차 스택으로 구성된다.

2. 3개의 합성곱을 사용하는 잔차 블록 생성
   - residual_block()

      ![잔차 블럭](image/02-03-sidualStack.png) 

3. 잔차 블록이 모인 잔차 스택 생성
   - residual_stack()
   - 사용하는 합성곱 필터의 수 64, 128, 256, 512

4. 첫 번째 잔차 블록에서 사용하는 입력은 56*56 채널 64, 마지막 합성곱 층에서 입력 채널 수가 256개로 늘어난다.
   - (56,56,64) --> (25,25,256)
   - 채널의 수가 다르면 스킵 연결을 통해 입력을 울력과 더할 수 가 없다.
   - 첫 번째 잔차 블록의 스킵 연결에 합성곱층을 추가하여 채널의 수를 256개로 맞추어야한다. 
   - 1*1 합성곱 또는 점별 합성곱(pointwise convolution)은 공간 방향과 차원을 유지하면서 채널차원을 변경해준다.
      ![합성곱 스킵](image/02-03-03-conv_skip2.png) 

In [17]:
def build_stack(x):
    # 첫 번째 잔차 스택의 첫 번째 잔차 블록만 스트라이드 1을 사용한다.
    x = residual_stack(x, 3, 64, first_stride=1)
    
    # 두 번째 ~ 네 번째 잔차 블록을 만든다.
    for blocks, filters in [(3, 64), (4, 128), (6, 256), (3, 512)]:
        # 잔차 스택 생성
        x = residual_stack(x, blocks, filters,first_stride=2)
    return x

def residual_stack(x, blocks, filters, first_stride=2):
    # 첫 번째 잔차 블록은 합성곱 스킵 연결을 사용
    # 첫 번째 잔차 블록의  첫 번째 합성곱 스트라이드는 first_stride
    x = residual_block(x, filters, first_stride=first_stride, conv_skip=True)
    for _ in range(blocks):
        # 나머지 잔차 블록의 첫 번째 합성곱 스트라이드는 1
        # 나머지 잔차 블록은 합성곱 스킵 연결을 사용하지 않음
        x = residual_block(x, filters, first_stride=1, conv_skip=False)
    return x

def residual_block(x, filters, first_stride=1, conv_skip=False):
    # 스킵 연결을 위해 입력 x를 따로 저장
    skip_conn = x
    
    # 첫 번째 1*1, filters개 필터, stride는 매개변수로 전달(1또는 2)
    # 필터 크기 1은 입력의 높이와 넓이를 변화시키지 않음
    x = layers.Conv2D(filters, kernel_size=1, strides=first_stride)(x)
    
    # 배치 정규화 층에 입력 데이터 x 전달
    # epsilon=1e-5: 분모가 0이 되는 것을 방지하기 위한 아주 작은 값을 분산에 더함
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation('relu')(x)
    
    # 두 번째 3*3, filters개 필터, 필터 크기 3, 스트라이드 1 합성곱 층에 입력 데이터 x 전달
    # 입력과 출력의 높이가 같아지도록 padding='same' 사용
    x = layers.Conv2D(filters, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation('relu')(x)
    
    # 세 번째 1*1, filters*4개 필터, 필터 크기 1, 스트라이드 1 합성곱 층에 입력 데이터 x 전달
    # 필터 크기 1은 입력의 높이와 넓이를 변화시키지 않음
    x = layers.Conv2D(filters * 4, kernel_size=1)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    
    if conv_skip == True:
        # 1 * 1 합성곱을 사용해 채널 크기를 filters * 4로 맞춤
        skip_conn = layers.Conv2D(filters * 4, kernel_size=1, strides=first_stride)(skip_conn)
        skip_conn = layers.BatchNormalization(epsilon=1e-5)(skip_conn)
    x = layers.Add()([skip_conn,x])
    x = layers.Activation('relu')(x)
    return x
    
    

#### 2.3 ReNet 모델 만들기 <a class="anchor" id="section_2_3"></a>
1. 전역 평균 풀링
   - 특성 맵의 공간 차원을 하나의 편균 값이나 최댓값으로 계산을 하는 연산.
   - 합성곱층에서 추출한 각각의 특성을 하나의 값으로 요약
   - 후속층의 파라미터 개수를 줄여 계산의 양을 감소
   - (7, 7, 2048) --> (2048,)

![ReNet 모델구조](image/02-03-02-ReNet2.png)    

In [18]:
# 잔차 스택과 잔차 블록을 쌓는다.
x = build_stack(x);

# 전역 평균 풀링 층에 입력 데이터 x 전달
# 출력 크기 : (512 * 4,) = (2048,)
x = layers.GlobalAveragePooling2D()(x)

# 1000개 뉴런, 소프트맥스 활성화 함수를 사용하는 밀집층에 입력 데이터 x 전달
# 출력 크기 : (1000,)
outputs = layers.Dense(1000, activation='softmax')(x)

model = keras.Model(inputs, outputs)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_6    │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │      9,472 │ zero_padding2d_6… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_7    │ (None, 114, 114,  │          0 │ activation_3[0][… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 56, 56,    │          0 │ zero_padding2d_7… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 56, 56,    │      4,160 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_19[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_15       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 56, 56,    │     36,928 │ activation_15[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_20[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_16       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 56, 56,    │     16,640 │ max_pooling2d_3[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 56, 56,    │     16,640 │ activation_16[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_21[0][0] 

 Total params: 31,937,256 (121.83 MB)

 Trainable params: 31,869,032 (121.57 MB)

 Non-trainable params: 68,224 (266.50 KB)

### Chapter 3 강아지와 고양이 사진 분류하기 <a class="anchor" id="chapter3"></a>
1. 케라스에는 이미지넷 데이터셋을 훈련한 ReNet 모델이 포함되어 있다.

In [20]:
from PIL import Image
import numpy as np  
from keras.applications import resnet 

dog_png = Image.open('./cat-dog-images/images/dog.png')

# resnet50 모델에 맞게 전처리
resnet_prep_dog = resnet.preprocess_input(np.array(dog_png))

resnet50 = keras.applications.ResNet50()

# dog_png 이미지에 대한 예측
predictions = resnet50.predict(resnet_prep_dog[np.newaxis, ...])

# 예측 결과 디코딩
resnet.decode_predictions(predictions, top=3)

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


[[('n02099712', 'Labrador_retriever', np.float32(0.38535184)),
  ('n02099601', 'golden_retriever', np.float32(0.08969971)),
  ('n02100735', 'English_setter', np.float32(0.042124238))]]

In [21]:
cat_png = Image.open('./cat-dog-images/images/cat.png')

# resnet50 모델에 맞게 전처리
resnet_prep_cat = resnet.preprocess_input(np.array(cat_png))

# cat_png 이미지에 대한 예측
predictions = resnet50.predict(resnet_prep_cat[np.newaxis, ...])

# 예측 결과 디코딩
resnet.decode_predictions(predictions, top=3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


[[('n02123045', 'tabby', np.float32(0.8686101)),
  ('n02124075', 'Egyptian_cat', np.float32(0.050774965)),
  ('n02123159', 'tiger_cat', np.float32(0.042567052))]]